# Searchlight analysis: think outside the box

Recall that we think of each $\beta$ as a point in high dimensional voxel space. We have to choose the voxels in which we want to do the MVPA for two main reasons:
   
1. **Cog neuro assumes the brain is made up of functional regions.** Based on the relevant unit of analysis (local regions). This idea about the brain as a modular structure comes from Cognitive Neuroscience.
2. **Avoid overfitting.** From a statistical point of view, we prefer to have as few dimensions as possible. As such, we'd like to reduce the noise in our dataset by only including the most relevant voxels.
   
Rule of thumb: we usually take ±80-100 voxels

<img src="http://drive.google.com/uc?export=view&id=1andZMeSCqfIQSfr7QwIRoYfHP0z7dGTD" style="height:150px"/>


## Searchlight analysis

from ([Joset et al., 2013](https://linkinghub.elsevier.com/retrieve/pii/S1053811913002917))

- a type of Multivoxel Pattern Analysis (MVPA), sometimes referred to as *information mapping*.
- a *searchlight* is a spatial moving window (kernel) that exhaustively searches the brain to localise representations. SA produces maps by measuring the information (read: variation in signal activity) in small spheres around each voxel.

## Searchlight workflow

- The two main things that determine the speed of a searchlight: the **kernel algorithm** and the **amount of parallelization**.
- Rule of thumb: start your searchlight analysis small!

### 1. Prepare the data



### 2. Set the searchlight parameters
### 3. Create the searchlight object
### 4. Create our classification kernel
### 5. Execute the searchlight

### Steps

1. Create a mask of one voxel and run the searchlight interactively to check whether the code works.
2. Use timestamps to extract the execution time
3. Print the number of voxels that are passed to the searchlight function
4. Run the searchlight as a job on the smallest unit of real data you have (a single run or single participant)
5. Check the runtime and memory usage of this searchlight (e.g. on slurm: `sacct -j $JID --format=jobid,maxvmsize,elapsed`).

In [5]:
import h5py
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
# Import libraries
import nibabel as nib
import numpy as np
import os 
import time
from nilearn import plotting
from brainiak.searchlight.searchlight import Searchlight
from brainiak.fcma.preprocessing import prepare_searchlight_mvpa_data
from brainiak import io
from pathlib import Path
from shutil import copyfile
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d 
import seaborn as sns 
import pandas as pd
from importlib import reload 

# Import machine learning libraries
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.svm import SVC

# import own functions
import utils
from utils import decode_variable, get_in_shape_levels, get_in_shape_blocks, plot_sub_isc_statmap
reload(utils)

%autosave 30
%matplotlib inline
sns.set(style = 'white', context='talk', font_scale=1, rc={"lines.linewidth": 2})

Autosaving every 30 seconds


## 1. Prepare the data

- want in form `[x,y,z,intensity]`


In [7]:
num_subjects = 8

B_data_blocks = []
mask_data_blocks = []
Vmask_data_blocks = []
names_data_blocks = []

for i in range(num_subjects):
    idx = i+1
    
    # change filename to subject #
    data_dir = '/Users/Daphne/Desktop/beta_series_smooth/'
    filename = 'beta_series_glm1_subjk_smooth.mat'
    filename = filename.replace('k', str(idx))
    
    subject = h5py.File(data_dir+filename,'r') 
    #print(list(subject.keys()))
    print(f'Get data for subject {idx}')
    # load and save data for respective subject
    B = subject['B'].value
    mask = subject['mask'].value
    Vmask = subject['Vmask']
    
    # === decode level names ===
    names = decode_variable(data_dir+filename, 'names')
    
    # append to lists
    B_data_blocks.append(B)
    mask_data_blocks.append(mask)
    Vmask_data_blocks.append(Vmask)
    names_data_blocks.append(names)

Get data for subject 1
Get data for subject 2
Get data for subject 3
Get data for subject 4
Get data for subject 5
Get data for subject 6
Get data for subject 7
Get data for subject 8


In [24]:

coords = np.where(mask_data_blocks[0] == 1)

B_data_blocks[0].shape

(220075, 18)

(79, 95, 79)

## Quicklinks & resources 

[brainiak searchlight tutorial](https://brainiak.org/tutorials/07-searchlight/)

[brainiak searchlight package](https://brainiak.org/docs/brainiak.searchlight.html#module-brainiak.searchlight)

### From `nilearn`

- [Searchlight with nilearn](http://nilearn.github.io/auto_examples/02_decoding/plot_haxby_searchlight.html#sphx-glr-auto-examples-02-decoding-plot-haxby-searchlight-py)